## Import Packages

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold


In [2]:
df = pd.read_csv('../data/interim/2_data_engineered.csv')

In [3]:
df.head()

,HR_before,HR_after,sp_total_distance,sp_average_distance,sp_median_distance,sp_total_angle,sp_average_angle,sp_median_angle,row_id,total_latency,mean_latency,median_latency
0,72.0,120.0,40146.187277,33.427300,30.113397,118.172835,0.098395,0.056699,1,10647.0,819.000000,500.0
1,84.0,134.0,32685.571041,27.215296,25.021036,204.802680,0.170527,0.090627,3,10912.0,496.000000,458.5
2,94.0,146.0,44043.556311,36.672403,35.104838,131.801018,0.109743,0.060777,5,13672.0,683.600000,583.0
3,83.0,132.0,28703.569543,23.899725,16.529260,101.726948,0.084702,0.033269,9,12038.0,573.238095,541.0
4,70.0,132.0,28257.159878,23.528027,22.943240,115.369011,0.096061,0.057553,12,12912.0,645.600000,579.0


## Choose your Feature!

In [4]:
df = df[['HR_before','HR_after','sp_median_distance','sp_median_angle','median_latency']]

## Train Test Split

In [5]:
# Assuming 'df' is your DataFrame and 'target_column' is the name of the target variable
X = df.drop(columns=['HR_after'])  # Features (drop the target column)
y = df['HR_after']                 # Target (the column you're predicting)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Define Evaluation Metrics Function

In [6]:
def evaluate_model(model, model_name, X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform 5-fold cross-validation
    cv_scores = cross_validate(model, X, y, cv=kfold, scoring=['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_percentage_error'])

    # Extracting individual metrics from the results
    rmse_scores = -cv_scores['test_neg_root_mean_squared_error']  # Negate to make positive
    r2_scores = cv_scores['test_r2']
    mape_scores = -cv_scores['test_neg_mean_absolute_percentage_error']  # Negate to make positive

    return [model_name, round(float(rmse_scores.mean()),5), round(float(r2_scores.mean()),5), round(float(mape_scores.mean())*100,5)]

## Modelling

### Linear Regression

In [ ]:
# Initialize the Linear Regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X_train, y_train)

# Make predictions on the test data
y_pred_lr = lr.predict(X_test)

lr_scores = evaluate_model(lr, 'Linear Regression', X, y)

## Random Forest

In [8]:
rf = RandomForestRegressor(random_state=42)

# rf.fit(X_train, y_train)

# y_pred_lr = rf.predict(X_test)

rf_scores = evaluate_model(rf, 'Random Forest Regressor', X, y)

## XGBoost

In [9]:
# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(random_state=42)

xgb_scores = evaluate_model(xgb_model, 'XGBoost Model', X, y)

## Put into dataframe

In [10]:
scores = [lr_scores, rf_scores, xgb_scores]
scores = pd.DataFrame(scores, columns=['Model Name', 'RMSE', 'R2', 'MAPE'])
scores

,Model Name,RMSE,R2,MAPE
0,Linear Regression,9.49464,-1.10511,6.33866
1,Random Forest Regressor,10.48184,-1.55527,6.84626
2,XGBoost Model,10.49842,-1.41772,7.06523
